Importation

In [58]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler


Pipeline de traitement pour dataset 1

In [ ]:
# Define the processing pipeline 1
def merge_rows_with_wheat_shift(df):
    """
    First shifts rows where wheat appears in longitude, then merges all pairs of rows.
    Properly preserves all values during the shifting process.
    """
    processed_rows = []
    
    i = 0
    while i < len(df) - 1:  # Process pairs of rows
        # Get current pair of rows
        numeric_row = df.iloc[i].copy()
        categorical_row = df.iloc[i + 1].copy()
        
        # Check if this is a wheat row that needs shifting
        if pd.notna(categorical_row['longitude']) and str(categorical_row['longitude']).strip().lower() == 'wheat':
            # For wheat rows:
            # 1. Move the values one column to the right
            categorical_row['city'] = categorical_row['soil']
            categorical_row['soil'] = categorical_row['crop']
            categorical_row['crop'] = 'WHEAT'  # Set crop to WHEAT
            
            # 2. Keep the numeric longitude from the first row if it exists
            if pd.isna(numeric_row['longitude']):
                numeric_row['longitude'] = np.nan  # Set as missing value when no longitude exists
        else:
            # For non-wheat rows:
            # If there's a longitude in the categorical row, use it
            if pd.notna(categorical_row['longitude']):
                numeric_row['longitude'] = categorical_row['longitude']
        
        # Merge categorical values into the numeric row
        categorical_cols = ['month', 'crop', 'soil', 'city']
        for col in categorical_cols:
            if pd.notna(categorical_row[col]):
                numeric_row[col] = categorical_row[col].strip()
        
        processed_rows.append(numeric_row)
        i += 2
    
    # If there's a lone last row, add it
    if i == len(df) - 1:
        processed_rows.append(df.iloc[-1])
    
    result_df = pd.DataFrame(processed_rows, columns=df.columns)
    
    # Convert longitude to numeric, replacing any remaining NaN with mean
    result_df['longitude'] = pd.to_numeric(result_df['longitude'], errors='coerce')
    result_df['longitude'] = result_df['longitude'].fillna(result_df['longitude'].mean())
    
    return result_df


def clean_categorical_values(df):
    """Standardizes categorical values by converting to uppercase and removing extra spaces."""
    categorical_cols = ['month', 'crop', 'soil', 'city']
    for col in categorical_cols:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip().str.upper()
    return df

def handle_missing_values(df):
    """Handles missing values in both numeric and categorical columns."""
    # For numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())
    
    # For categorical columns
    categorical_cols = ['month', 'crop', 'soil', 'city']
    for col in categorical_cols:
        mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else "UNKNOWN"
        df[col] = df[col].fillna(mode_value)
    
    return df

def create_features(df):
    """Creates new features from existing data."""
    # Temperature ratio (with handling for zero values)
    df['temp_ratio'] = df['Max Temp'] / df['Min Temp'].replace(0, np.nan)
    df['temp_ratio'] = df['temp_ratio'].fillna(df['temp_ratio'].median())
    
    # Humidity/radiation ratio
    df['humidity_rad_ratio'] = df['Humidity'] / df['Rad'].replace(0, np.nan)
    df['humidity_rad_ratio'] = df['humidity_rad_ratio'].fillna(df['humidity_rad_ratio'].median())
    
    # Season mapping
    season_mapping = {
        'DECEMBER': 'WINTER', 'JANUARY': 'WINTER', 'FEBRUARY': 'WINTER',
        'MARCH': 'SPRING', 'APRIL': 'SPRING', 'MAY': 'SPRING',
        'JUNE': 'SUMMER', 'JULY': 'SUMMER', 'AUGUST': 'SUMMER',
        'SEPTEMBER': 'AUTUMN', 'OCTOBER': 'AUTUMN', 'NOVEMBER': 'AUTUMN'
    }
    df['season'] = df['month'].map(season_mapping)
    
    return df

def normalize_and_encode(df):
    """Normalizes numeric features and encodes categorical variables."""
    df_encoded = df.copy()
    
    # Normalize numeric columns
    numeric_cols = ['water req', 'Min Temp', 'Max Temp', 'Humidity', 'Wind', 
                   'Sun', 'Rad', 'Rain', 'altitude', 'latitude', 'longitude',
                   'temp_ratio', 'humidity_rad_ratio']
    
    # Create and fit scaler
    scaler = MinMaxScaler()
    df_encoded[numeric_cols] = scaler.fit_transform(df_encoded[numeric_cols])
    
    # Encode categorical columns
    categorical_cols = ['month', 'crop', 'soil', 'city', 'season']
    encoders = {}
    for col in categorical_cols:
        encoders[col] = LabelEncoder()
        df_encoded[col] = encoders[col].fit_transform(df_encoded[col])
    
    return df_encoded, encoders

def process_dataset1(input_file, output_dir):
    """Main function to process the dataset."""
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Apply the processing pipeline
    df = merge_rows_with_wheat_shift(df)
    df = clean_categorical_values(df)
    df = handle_missing_values(df)
    df = create_features(df)
    
    # Save the preprocessed but non-normalized version
    df.to_csv(os.path.join(output_dir, 'dataset_1_preprocessed.csv'), index=False)
    
    # Create normalized version
    df_normalized, encoders = normalize_and_encode(df)
    
    # Save the normalized version
    df_normalized.to_csv(os.path.join(output_dir, 'dataset_1_normalized.csv'), index=False)
    
    return df, df_normalized, encoders

In [ ]:
# Utilisation du pipeline 1
if __name__ == "__main__":

    input_file = r"Datasets2\dataset1\data_plants.csv" 
    
    output_dir = "Output"
    
    # Unpack all three returned values
    df_raw, df_normalized, encoders = process_dataset1(input_file, output_dir)

Pipeline de traitement pour dataset 2

In [10]:
# Define the processing pipeline 2
def convert_timestamp(df):
    """Converts Unix timestamps to datetime and extracts temporal features."""
    # Convert Unix timestamp to datetime
    df['datetime'] = pd.to_datetime(df['time'], unit='s')
    
    # Extract temporal features
    df['date'] = df['datetime'].dt.date
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['year'] = df['datetime'].dt.year
    df['day_of_week'] = df['datetime'].dt.dayofweek
    
    # Drop original time column and datetime (keep date as string)
    df['date'] = df['date'].astype(str)
    df = df.drop(['time', 'datetime'], axis=1)
    
    return df

def categorize_hour(df):
    """Categorizes hours into periods of the day."""
    conditions = [
        (df['hour'] >= 5) & (df['hour'] < 12),
        (df['hour'] >= 12) & (df['hour'] < 17),
        (df['hour'] >= 17) & (df['hour'] < 21),
        (df['hour'] >= 21) | (df['hour'] < 5)
    ]
    periods = ['MORNING', 'AFTERNOON', 'EVENING', 'NIGHT']
    
    df['day_period'] = np.select(conditions, periods, default='UNKNOWN')
    return df

def handle_missing_values(df):
    """Handles any missing values in the dataset."""
    # For numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())
    
    # For categorical columns (if any were created)
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else "UNKNOWN"
        df[col] = df[col].fillna(mode_value)
    
    return df

def normalize_and_encode(df):
    """Normalizes numeric features while preserving categorical ones."""
    # Identify numeric columns to normalize
    numeric_cols = ['water', 'hour']
    
    # Create and fit scaler
    scaler = MinMaxScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    
    # encode categorical columns
    categorical_cols = ['day_period']
    encoders = {}
    for col in categorical_cols:
        encoders[col] = LabelEncoder()
        df[col] = encoders[col].fit_transform(df[col])
    
    return df, scaler

def process_dataset2(input_file, output_dir):
    """Main function to process the tomato dataset."""
    try:
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Try to read first few lines to check for header
        with open(input_file, 'r') as f:
            first_line = f.readline().strip()
        
        # If first line contains header-like content, skip it
        if 'time' in first_line.lower() or 'simulation' in first_line.lower():
            df = pd.read_csv(input_file, skiprows=1, names=['simulation_id', 'time', 'water', 'hour'])
        else:
            df = pd.read_csv(input_file, names=['simulation_id', 'time', 'water', 'hour'])
        
        # Apply the processing pipeline
        df = convert_timestamp(df)
        df = categorize_hour(df)
        df = handle_missing_values(df)
        
        # Save the preprocessed but non-normalized version
        df.to_csv(os.path.join(output_dir, 'dataset_2_preprocessed.csv'), index=False)
        
        # Create normalized version
        df_normalized, scaler = normalize_and_encode(df)
        
        # Save the normalized version
        df_normalized.to_csv(os.path.join(output_dir, 'dataset_2_normalized.csv'), index=False)
        
        return df, df_normalized, scaler
        
    except Exception as e:
        print(f"Error processing dataset: {str(e)}")
        raise


In [11]:
# Utilisation du pipeline 2
if __name__ == "__main__":
    input_file = r"Datasets2/dataset2/tomates.csv"
    output_dir = "Output"
    
    # Process the dataset
    df_raw, df_normalized, scaler = process_dataset2(input_file, output_dir)

Pipeline de traitement pour dataset 3

In [30]:
from typing import Tuple, Optional, List
import logging

In [34]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Constants
REQUIRED_COLUMNS = {
    'growing_period': ['Crop', 'Total growing period (days)'],
    'water_need': ['Crop', 'Crop water need (mm/total growing period)']
}

def split_range(range_str: str) -> Tuple[float, float]:
    """
    Split a range string formatted as "min-max" into min and max values as floats.
    
    Args:
        range_str: String containing the range in format "min-max"
        
    Returns:
        Tuple of (min_value, max_value) as floats. Returns (NaN, NaN) if invalid.
    """
    try:
        # Handle various formats and clean the input
        range_str = str(range_str).strip().replace(' ', '')
        
        # Handle single values
        if range_str.replace('.', '').isdigit():
            value = float(range_str)
            return value, value
            
        # Handle range values
        if '-' in range_str:
            parts = range_str.split('-')
            if len(parts) == 2:
                min_val = float(parts[0])
                max_val = float(parts[1])
                # Ensure min <= max
                if min_val <= max_val:
                    return min_val, max_val
                
        return np.nan, np.nan
    except Exception as e:
        logging.warning(f"Error processing range '{range_str}': {str(e)}")
        return np.nan, np.nan

def process_ranges(df: pd.DataFrame, col: str, new_min_col: str, new_max_col: str) -> pd.DataFrame:
    """
    Process a column containing range strings into separate min and max columns.
    """
    if col not in df.columns:
        logging.error(f"Column '{col}' not found in DataFrame")
        return df
        
    ranges = df[col].apply(split_range)
    df[new_min_col] = ranges.apply(lambda x: x[0])
    df[new_max_col] = ranges.apply(lambda x: x[1])
    
    return df

def expand_crop_names(crop_name: str) -> List[str]:
    """
    Split compound crop names into individual crops.
    
    Args:
        crop_name: String containing possibly multiple crop names separated by '/'
        
    Returns:
        List of individual crop names
    """
    return [name.strip().upper() for name in crop_name.split('/')]

def process_dataset3(input_file: str, output_dir: str) -> Optional[pd.DataFrame]:
    """
    Process the FAO Website_data.xls dataset with improved crop name handling.
    """
    try:
        # Validate input file
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"Input file not found: {input_file}")
            
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Try reading as tab-delimited file first
        logging.info(f"Reading input file as tab-delimited: {input_file}")
        try:
            df = pd.read_csv(input_file, sep='\t')
        except Exception as e:
            logging.warning(f"Failed to read as tab-delimited, trying Excel format: {str(e)}")
            df = pd.read_excel(input_file, engine='xlrd')
        
        # Split the dataframe into two parts based on non-null values
        df_period = df[['Crop', 'Total growing period (days)']].dropna(subset=['Total growing period (days)'])
        df_water = df[['Crop', 'Crop water need (mm/total growing period)']].dropna(subset=['Crop water need (mm/total growing period)'])
        
        # Rename columns for consistency
        df_period = df_period.rename(columns={
            'Total growing period (days)': 'Total_growing_period'
        })
        
        df_water = df_water.rename(columns={
            'Crop water need (mm/total growing period)': 'Crop_water_need'
        })
        
        # Clean crop names
        df_period['Crop'] = df_period['Crop'].str.strip().str.upper()
        df_water['Crop'] = df_water['Crop'].str.strip().str.upper()
        
        # Expand compound crop names
        period_rows = []
        for _, row in df_period.iterrows():
            for crop in expand_crop_names(row['Crop']):
                new_row = row.copy()
                new_row['Crop'] = crop
                period_rows.append(new_row)
        df_period = pd.DataFrame(period_rows)
        
        water_rows = []
        for _, row in df_water.iterrows():
            for crop in expand_crop_names(row['Crop']):
                new_row = row.copy()
                new_row['Crop'] = crop
                water_rows.append(new_row)
        df_water = pd.DataFrame(water_rows)
        
        # Process range values for both dataframes
        df_period = process_ranges(
            df_period,
            'Total_growing_period',
            'Total_growing_period_min',
            'Total_growing_period_max'
        )
        df_period = df_period.drop('Total_growing_period', axis=1)
        
        df_water = process_ranges(
            df_water,
            'Crop_water_need',
            'Crop_water_need_min',
            'Crop_water_need_max'
        )
        df_water = df_water.drop('Crop_water_need', axis=1)
        
        # Calculate averages for both dataframes
        df_period['Total_growing_period_avg'] = df_period[
            ['Total_growing_period_min', 'Total_growing_period_max']
        ].mean(axis=1)
        
        df_water['Crop_water_need_avg'] = df_water[
            ['Crop_water_need_min', 'Crop_water_need_max']
        ].mean(axis=1)
        
        # Merge the dataframes on Crop name
        df_combined = pd.merge(
            df_period,
            df_water,
            on='Crop',
            how='outer'
        )
        
        # Sort by crop name for better readability
        df_combined = df_combined.sort_values('Crop')
        
        # Save processed data
        output_file = os.path.join(output_dir, 'dataset_3_processed.csv')
        df_combined.to_csv(output_file, index=False)
        logging.info(f"Processed data saved to: {output_file}")
        
        return df_combined
        
    except Exception as e:
        logging.error(f"Error processing dataset: {str(e)}")
        return None


In [35]:

if __name__ == "__main__":
    input_file = r"Datasets2/dataset3/Website_data.xls"
    output_dir = "Output"
    
    try:
        df_processed = process_dataset3(input_file, output_dir)
        if df_processed is not None:
            logging.info("Dataset processing completed successfully")
            logging.info(f"Processed {len(df_processed)} records")
        else:
            logging.error("Dataset processing failed")
    except Exception as e:
        logging.error(f"Pipeline execution failed: {str(e)}")

2025-02-21 18:02:18,848 - INFO - Reading input file as tab-delimited: Datasets2/dataset3/Website_data.xls
2025-02-21 18:02:18,891 - INFO - Processed data saved to: Output\dataset_3_processed.csv
2025-02-21 18:02:18,893 - INFO - Dataset processing completed successfully
2025-02-21 18:02:18,895 - INFO - Processed 40 records


Pipeline de traitement pour dataset 4

In [38]:

# Constants
CROP_TYPE_MAPPING = {
    1: 'Paddy',
    2: 'Ground Nuts'
}

def validate_data(df: pd.DataFrame) -> bool:
    """
    Validate the input dataframe structure and content.
    """
    required_columns = [
        'CropType', 'CropDays', 'Soil Moisture', 'Soil Temperature',
        'Temperature', 'Humidity', 'Irrigation(Y/N)'
    ]
    
    try:
        # Check for required columns
        for col in required_columns:
            if col not in df.columns:
                logging.error(f"Missing required column: {col}")
                return False
        
        # Validate data types
        if not df['CropType'].dtype in ['int64', 'float64']:
            logging.error("CropType column should contain numeric values")
            return False
            
        if not df['Irrigation(Y/N)'].dtype in ['int64', 'float64']:
            logging.error("Irrigation column should contain numeric values")
            return False
            
        return True
        
    except Exception as e:
        logging.error(f"Validation error: {str(e)}")
        return False

def process_dataset4(input_file: str, output_dir: str) -> Optional[pd.DataFrame]:
    """
    Process the irrigation dataset.
    """
    try:
        # Validate input file
        if not os.path.exists(input_file):
            raise FileNotFoundError(f"Input file not found: {input_file}")
            
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Read only the main data part (first 7 columns)
        logging.info(f"Reading input file: {input_file}")
        df = pd.read_excel(input_file, usecols=range(7))
        
        # Validate data structure
        if not validate_data(df):
            raise ValueError("Data validation failed")
        
        # Clean column names
        df.columns = df.columns.str.strip()
        
        # Convert crop types to names
        df['CropType'] = df['CropType'].map(CROP_TYPE_MAPPING)
        
        # Save processed data
        output_file = os.path.join(output_dir, 'dataset_4_processed.csv')
        df.to_csv(output_file, index=False)
        logging.info(f"Processed data saved to: {output_file}")
        
        return df
        
    except Exception as e:
        logging.error(f"Error processing dataset: {str(e)}")
        return None


In [39]:

if __name__ == "__main__":
    input_file = r"Datasets2/dataset4/Project_datasheet_2019-2020.xlsx"
    output_dir = "Output"
    
    try:
        df_processed = process_dataset4(input_file, output_dir)
        if df_processed is not None:
            logging.info("Dataset processing completed successfully")
            logging.info(f"Processed {len(df_processed)} records")
        else:
            logging.error("Dataset processing failed")
    except Exception as e:
        logging.error(f"Pipeline execution failed: {str(e)}")

2025-02-21 22:23:17,446 - INFO - Reading input file: Datasets2/dataset4/Project_datasheet_2019-2020.xlsx
2025-02-21 22:23:17,491 - INFO - Processed data saved to: Output\dataset_4_processed.csv
2025-02-21 22:23:17,491 - INFO - Dataset processing completed successfully
2025-02-21 22:23:17,497 - INFO - Processed 150 records


Pipeline de traitement pour dataset 6

In [42]:
def get_season(month: str) -> str:
    """
    Convert month to season.
    """
    month = month.lower()
    seasons = {
        'winter': ['december', 'january', 'february'],
        'spring': ['march', 'april', 'may'],
        'summer': ['june', 'july', 'august'],
        'autumn': ['september', 'october', 'november']
    }
    
    for season, months in seasons.items():
        if month in months:
            return season.upper()
    return 'UNKNOWN'

def read_and_process_file(file_path: str) -> Optional[pd.DataFrame]:
    """
    Read and process a single data file.
    """
    try:
        # Determine file type and read accordingly
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            logging.error(f"Unsupported file format: {file_path}")
            return None
        
        # Standardize column names to lowercase
        df.columns = df.columns.str.lower().str.strip()
        
        return df
    
    except Exception as e:
        logging.error(f"Error reading file {file_path}: {str(e)}")
        return None

def process_dataset6(input_dir: str, output_dir: str) -> Optional[pd.DataFrame]:
    """
    Process the agricultural dataset.
    """
    try:
        # Define input files
        files = {
            'ble': 'blé.csv',
            'riz': 'le riz2.csv',
            'mais': 'maïs.xlsx',
            'potato': 'potato.csv'
        }
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Read and combine all files
        dataframes = []
        for name, file in files.items():
            file_path = os.path.join(input_dir, file)
            df = read_and_process_file(file_path)
            if df is not None:
                dataframes.append(df)
            else:
                logging.error(f"Failed to process {file}")
                return None
        
        # Combine all dataframes
        df_combined = pd.concat(dataframes, ignore_index=True)
        
        # Standardize column names and remove duplicates
        df_combined.columns = df_combined.columns.str.lower().str.strip()
        
        # Keep only unique columns
        df_combined = df_combined.loc[:, ~df_combined.columns.duplicated()]
        
        # Convert categorical fields to uppercase
        categorical_columns = ['month', 'crop', 'soil', 'city']
        for col in categorical_columns:
            df_combined[col] = df_combined[col].str.upper()
        
        # Convert numeric columns to appropriate types
        numeric_columns = ['water req', 'min temp', 'max temp', 'humidity', 'wind', 
                         'sun', 'rad', 'rain', 'altitude', 'latitude', 'longitude']
        for col in numeric_columns:
            df_combined[col] = pd.to_numeric(df_combined[col], errors='coerce')
        
        # Add new features
        # 1. Temperature ratio
        df_combined['temp_ratio'] = df_combined['max temp'] / df_combined['min temp']
        
        # 2. Humidity-radiation ratio
        df_combined['humidity_rad_ratio'] = df_combined['humidity'] / df_combined['rad']
        
        # 3. Season
        df_combined['season'] = df_combined['month'].apply(get_season)
        
        # Reorder columns for better readability
        columns_order = [
            'water req', 'month', 'min temp', 'max temp', 'humidity', 'wind', 
            'sun', 'rad', 'rain', 'altitude', 'latitude', 'longitude',
            'crop', 'soil', 'city', 'temp_ratio', 'humidity_rad_ratio', 'season'
        ]
        df_combined = df_combined[columns_order]
        
        # Save processed data
        output_file = os.path.join(output_dir, 'dataset_6_processed.csv')
        df_combined.to_csv(output_file, index=False)
        logging.info(f"Processed data saved to: {output_file}")
        
        return df_combined
        
    except Exception as e:
        logging.error(f"Error processing dataset: {str(e)}")
        return None

In [43]:
# Utilisation du pipeline 6
if __name__ == "__main__":
    input_dir = r"Datasets2/Dataset6/riz+mais+patate+ble"
    output_dir = "Output"
    
    # Configure logging
    logging.basicConfig(level=logging.INFO)
    
    try:
        df_processed = process_dataset6(input_dir, output_dir)
        if df_processed is not None:
            logging.info("Dataset processing completed successfully")
            logging.info(f"Processed {len(df_processed)} records")
            
            # Print sample of processed data
            logging.info("\nSample of processed data:")
            logging.info(df_processed.head())
            
            # Print data quality information
            logging.info("\nColumn information:")
            for col in df_processed.columns:
                null_count = df_processed[col].isnull().sum()
                logging.info(f"{col}: {df_processed[col].dtype} (null values: {null_count})")
        else:
            logging.error("Dataset processing failed")
    except Exception as e:
        logging.error(f"Pipeline execution failed: {str(e)}")

2025-02-22 00:06:27,640 - INFO - Processed data saved to: Output\dataset_6_processed.csv
2025-02-22 00:06:27,641 - INFO - Dataset processing completed successfully
2025-02-22 00:06:27,642 - INFO - Processed 115 records
2025-02-22 00:06:27,644 - INFO - 
Sample of processed data:
2025-02-22 00:06:27,645 - INFO -    water req  month  min temp  max temp  humidity  wind  sun   rad   rain  \
0       46.8  MARCH      16.0      32.0        35   192  8.0  19.2    5.0   
1      181.3  APRIL      22.0      37.0        26   240  9.0  22.6    4.0   
2      327.5    MAY      26.0      41.0        27   288  9.0  23.4   18.0   
3      187.5   JUNE      27.0      39.0        46   312  7.0  20.6   51.0   
4        0.0   JULY      25.0      34.0        73   264  5.0  17.5  213.0   

   altitude  latitude  longitude   crop       soil    city  temp_ratio  \
0       431     26.91        NaN  WHEAT  RED SANDY  JAIPUR    2.000000   
1       431     26.91        NaN  WHEAT  RED SANDY  JAIPUR    1.681818   
2  

Preparing final dataset

In [55]:


def load_datasets():
    """Load and display initial information about datasets"""
    df1 = pd.read_csv('Output/dataset_1_preprocessed.csv')
    df6 = pd.read_csv('Output/dataset_6_processed.csv')
    df3 = pd.read_csv('Output/dataset_3_processed.csv')
    
    print("\nInitial Dataset Information:")
    print("Dataset 1 shape:", df1.shape)
    print("Dataset 6 shape:", df6.shape)
    print("Dataset 3 shape:", df3.shape)
    
    return df1, df6, df3

def standardize_column_names(df):
    """Standardize column names to lowercase and replace spaces with underscores"""
    return df.rename(columns=lambda x: x.lower().replace(' ', '_'))

def clean_and_standardize(df1, df6):
    """Clean and combine datasets 1 and 6"""
    # Make copies to avoid modifying original dataframes
    df1 = df1.copy()
    df6 = df6.copy()
    
    # Standardize column names
    df1 = standardize_column_names(df1)
    df6 = standardize_column_names(df6)
    
    # Ensure column names match exactly before concatenation
    common_columns = list(set(df1.columns) & set(df6.columns))
    df1 = df1[common_columns]
    df6 = df6[common_columns]
    
    # Combine datasets
    combined_df = pd.concat([df1, df6], ignore_index=True)
    
    # Convert month and crop to uppercase
    if 'month' in combined_df.columns:
        combined_df['month'] = combined_df['month'].str.upper()
    if 'crop' in combined_df.columns:
        combined_df['crop'] = combined_df['crop'].str.upper()
    
    print("\nCombined Dataset Information:")
    print("Shape after combination:", combined_df.shape)
    print("Columns:", combined_df.columns.tolist())
    
    return combined_df

def process_dataset3(df3):
    """Process dataset 3 for merging"""
    df3 = df3.copy()
    # Standardize column names
    df3 = standardize_column_names(df3)
    
    # Convert crop to uppercase
    if 'crop' in df3.columns:
        df3['crop'] = df3['crop'].str.upper()
    
    print("\nDataset 3 Processing Information:")
    print("Columns after processing:", df3.columns.tolist())
    
    return df3

def merge_datasets(combined_df, df3):
    """Merge the combined dataset with dataset 3"""
    # Ensure 'crop' column exists in both dataframes
    if 'crop' not in combined_df.columns or 'crop' not in df3.columns:
        raise ValueError("'crop' column missing from one or both datasets")
    
    # Merge datasets
    final_df = pd.merge(
        combined_df,
        df3,
        how='left',
        on='crop'
    )
    
    print("\nMerge Information:")
    print("Shape after merge:", final_df.shape)
    print("Columns after merge:", final_df.columns.tolist())
    
    return final_df

def select_features(df):
    """Select and verify relevant features"""
    # Define desired features
    selected_features = [
        'water_req',  # target variable
        'month',
        'min_temp',
        'max_temp',
        'humidity',
        'wind',
        'sun',
        'rad',
        'rain',
        'altitude',
        'latityde',
        'longitude',
        'soil',
        'city',
        'temp_ratio',
        'humidity_rad_ratio',
        'season',
        'crop',
        'total_growing_period_avg',
        'crop_water_need_min',
        'crop_water_need_max',
        'crop_water_need_avg'
    ]
    
    # Check which features are available
    available_features = [feat for feat in selected_features if feat in df.columns]
    missing_features = [feat for feat in selected_features if feat not in df.columns]
    
    print("\nFeature Selection Information:")
    print("Available features:", available_features)
    print("Missing features:", missing_features)
    
    # Select only available features
    final_df = df[available_features]
    
    return final_df

# def clean_final_dataset(df):
#     """Perform final cleaning on the dataset"""
#     # Drop rows with missing values
#     df_cleaned = df.dropna()
    
#     # Convert categorical variables to uppercase
#     categorical_cols = ['month', 'soil', 'season', 'crop']
#     for col in categorical_cols:
#         if col in df_cleaned.columns:
#             df_cleaned[col] = df_cleaned[col].str.upper()
    
#     print("\nFinal Cleaning Information:")
#     print("Shape before cleaning:", df.shape)
#     print("Shape after cleaning:", df_cleaned.shape)
    
#     return df_cleaned

def process_join_datasets():
    """Main function to process and join all datasets"""
    try:
        # Load datasets
        print("Loading datasets...")
        df1, df6, df3 = load_datasets()
        
        # Clean and combine datasets 1 and 6
        print("\nCleaning and standardizing datasets...")
        combined_df = clean_and_standardize(df1, df6)
        
        # Process dataset 3
        print("\nProcessing dataset 3...")
        df3_processed = process_dataset3(df3)
        
        # Merge datasets
        print("\nMerging datasets...")
        merged_df = merge_datasets(combined_df, df3_processed)
        
        # Select features
        print("\nSelecting relevant features...")
        final_df = select_features(merged_df)
        
        # Final cleaning
        # print("\nPerforming final cleaning...")
        # final_df = clean_final_dataset(final_df)
        
        # Save the final dataset
        output_path = 'Output/dataset_final_joined.csv'
        final_df.to_csv(output_path, index=False)
        print(f"\nProcess completed. Dataset saved as: {output_path}")
        
        # Print final statistics
        print("\nFinal Dataset Statistics:")
        print(f"Total number of rows: {len(final_df)}")
        print(f"Number of unique crops: {final_df['crop'].nunique()}")
        print(f"Final columns: {final_df.columns.tolist()}")
        print("\nSample of final dataset:")
        print(final_df.head())
        
        return final_df
        
    except Exception as e:
        print(f"\nError occurred: {str(e)}")
        raise



In [56]:
# # Execute the processing
# if __name__ == "__main__":
final_df = process_join_datasets()

Loading datasets...

Initial Dataset Information:
Dataset 1 shape: (1910, 18)
Dataset 6 shape: (115, 18)
Dataset 3 shape: (40, 7)

Cleaning and standardizing datasets...

Combined Dataset Information:
Shape after combination: (2025, 18)
Columns: ['longitude', 'humidity_rad_ratio', 'season', 'humidity', 'water_req', 'latitude', 'soil', 'rain', 'rad', 'month', 'crop', 'min_temp', 'wind', 'max_temp', 'temp_ratio', 'city', 'sun', 'altitude']

Processing dataset 3...

Dataset 3 Processing Information:
Columns after processing: ['crop', 'total_growing_period_min', 'total_growing_period_max', 'total_growing_period_avg', 'crop_water_need_min', 'crop_water_need_max', 'crop_water_need_avg']

Merging datasets...

Merge Information:
Shape after merge: (2025, 24)
Columns after merge: ['longitude', 'humidity_rad_ratio', 'season', 'humidity', 'water_req', 'latitude', 'soil', 'rain', 'rad', 'month', 'crop', 'min_temp', 'wind', 'max_temp', 'temp_ratio', 'city', 'sun', 'altitude', 'total_growing_period_

In [57]:
# missing values
final_df.isnull().sum()

water_req                      0
month                          0
min_temp                       0
max_temp                       0
humidity                       0
wind                           0
sun                            0
rad                            0
rain                           0
altitude                       0
longitude                     30
soil                           0
city                           0
temp_ratio                     0
humidity_rad_ratio             0
season                         0
crop                           0
total_growing_period_avg      27
crop_water_need_min         1330
crop_water_need_max         1330
crop_water_need_avg         1330
dtype: int64

data final preprocessing pipeline

In [59]:

def preprocess_data(df):
    """
    Preprocess the dataset with encoding and normalization
    """
    # Make a copy to avoid modifying the original dataframe
    df = df.copy()
    
    # Drop specified columns
    df = df.drop(['crop_water_need_min', 'crop_water_need_max', 'crop_water_need_avg'], axis=1)
    
    # Handle missing values
    df['longitude'] = df['longitude'].fillna(df['longitude'].median())
    df['total_growing_period_avg'] = df['total_growing_period_avg'].fillna(df['total_growing_period_avg'].median())
    
    # Separate numerical and categorical columns
    categorical_columns = ['month', 'soil', 'city', 'season', 'crop']
    numerical_columns = ['min_temp', 'max_temp', 'humidity', 'wind', 'sun', 'rad', 'rain', 
                        'altitude', 'longitude', 'temp_ratio', 'humidity_rad_ratio', 
                        'total_growing_period_avg']
    
    # Initialize dictionary to store encoders
    label_encoders = {}
    
    # Encode categorical variables
    for column in categorical_columns:
        label_encoders[column] = LabelEncoder()
        df[column] = label_encoders[column].fit_transform(df[column])
    
    # Create scaled version of numerical columns
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    
    # Separate features and target
    X = df.drop('water_req', axis=1)
    y = df['water_req']
    
    # Scale target variable
    y_scaler = StandardScaler()
    y = y_scaler.fit_transform(y.values.reshape(-1, 1)).ravel()
    
    print("\nPreprocessing Information:")
    print("Features shape:", X.shape)
    print("Target shape:", y.shape)
    print("\nFeature columns:", X.columns.tolist())
    
    return X, y, label_encoders, scaler, y_scaler

def split_data(X, y, test_size=0.2, random_state=42):
    """
    Split the data into training and testing sets
    """
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    
    print("\nData Split Information:")
    print("Training set shape:", X_train.shape)
    print("Testing set shape:", X_test.shape)
    
    return X_train, X_test, y_train, y_test

def create_preprocessed_datasets():
    """
    Create and save preprocessed datasets
    """
    try:
        # Load the joined dataset
        df = pd.read_csv('Output/dataset_final_joined.csv')
        
        print("Original dataset shape:", df.shape)
        
        # Preprocess the data
        X, y, label_encoders, scaler, y_scaler = preprocess_data(df)
        
        # Split the data
        X_train, X_test, y_train, y_test = split_data(X, y)
        
        # Save preprocessed datasets
        pd.DataFrame(X_train, columns=X.columns).to_csv('Output/X_train_preprocessed.csv', index=False)
        pd.DataFrame(X_test, columns=X.columns).to_csv('Output/X_test_preprocessed.csv', index=False)
        pd.DataFrame(y_train, columns=['water_req']).to_csv('Output/y_train_preprocessed.csv', index=False)
        pd.DataFrame(y_test, columns=['water_req']).to_csv('Output/y_test_preprocessed.csv', index=False)
        
        print("\nPreprocessed datasets have been saved to Output folder")
        
        # Save some statistics about the data
        print("\nData Statistics:")
        print("Number of features:", X.shape[1])
        print("Categorical features:", len([col for col in X.columns if X[col].dtype == 'int32']))
        print("Numerical features:", len([col for col in X.columns if X[col].dtype == 'float64']))
        
        return X_train, X_test, y_train, y_test, label_encoders, scaler, y_scaler
        
    except Exception as e:
        print(f"\nError occurred: {str(e)}")
        raise


In [60]:
# Execute preprocessing
X_train, X_test, y_train, y_test, label_encoders, scaler, y_scaler = create_preprocessed_datasets()

# Print sample of preprocessed data
print("\nSample of preprocessed training data:")
print(pd.DataFrame(X_train, columns=X_train.columns).head())
print("\nSample of preprocessed target values:")
print(pd.DataFrame(y_train, columns=['water_req']).head())

Original dataset shape: (2025, 21)

Preprocessing Information:
Features shape: (2025, 17)
Target shape: (2025,)

Feature columns: ['month', 'min_temp', 'max_temp', 'humidity', 'wind', 'sun', 'rad', 'rain', 'altitude', 'longitude', 'soil', 'city', 'temp_ratio', 'humidity_rad_ratio', 'season', 'crop', 'total_growing_period_avg']

Data Split Information:
Training set shape: (1620, 17)
Testing set shape: (405, 17)

Preprocessed datasets have been saved to Output folder

Data Statistics:
Number of features: 17
Categorical features: 5
Numerical features: 12

Sample of preprocessed training data:
      month  min_temp  max_temp  humidity      wind       sun       rad  \
1823      6  0.764613 -0.378992  1.148785 -0.858545 -1.165384 -1.029857   
680       6  0.740334  1.224983 -0.783840  1.149554 -0.042480  0.186248   
1773      5  0.407362  0.275430  0.242494  0.815312 -0.454514 -0.495650   
1302      7 -0.293265 -0.382200  0.580433 -0.189086  1.098534  1.116396   
1752      6  0.764613  0.894

Prediction

In [61]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [62]:
# Pipeline training and evaluation

def create_and_evaluate_models(X_train, X_test, y_train, y_test):
    """
    Create, train and evaluate multiple models for water requirement prediction
    """
    models = {}
    results = {}
    
    # 1. Random Forest Model
    rf_pipeline = Pipeline([
        ('rf', RandomForestRegressor(random_state=42))
    ])
    
    rf_params = {
        'rf__n_estimators': [100, 200, 300],
        'rf__max_depth': [10, 20, 30],
        'rf__min_samples_split': [2, 5],
        'rf__min_samples_leaf': [1, 2]
    }
    
    rf_grid = GridSearchCV(
        rf_pipeline,
        rf_params,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    
    print("Training Random Forest model...")
    rf_grid.fit(X_train, y_train)
    models['random_forest'] = rf_grid.best_estimator_
    
    # 2. Neural Network (Scikit-learn MLPRegressor)
    mlp_pipeline = Pipeline([
        ('mlp', MLPRegressor(random_state=42, max_iter=1000))
    ])
    
    mlp_params = {
        'mlp__hidden_layer_sizes': [(50,), (100,), (50, 25)],
        'mlp__activation': ['relu', 'tanh'],
        'mlp__learning_rate_init': [0.001, 0.01]
    }
    
    mlp_grid = GridSearchCV(
        mlp_pipeline,
        mlp_params,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    
    print("Training Neural Network (MLPRegressor) model...")
    mlp_grid.fit(X_train, y_train)
    models['neural_network_sklearn'] = mlp_grid.best_estimator_
    
    # 3. Deep Neural Network (TensorFlow)
    def create_deep_model(input_dim):
        model = Sequential([
            Dense(128, activation='relu', input_dim=input_dim),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer=Adam(learning_rate=0.001),
                     loss='mse',
                     metrics=['mae'])
        return model
    
    print("Training Deep Neural Network (TensorFlow) model...")
    dnn_model = create_deep_model(X_train.shape[1])
    dnn_model.fit(
        X_train,
        y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        verbose=0
    )
    models['deep_neural_network'] = dnn_model
    
    # Evaluate all models
    for name, model in models.items():
        if name == 'deep_neural_network':
            y_pred = model.predict(X_test, verbose=0)
        else:
            y_pred = model.predict(X_test)
        
        results[name] = {
            'mse': mean_squared_error(y_test, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
            'mae': mean_absolute_error(y_test, y_pred),
            'r2': r2_score(y_test, y_pred)
        }
    
    return models, results

def print_results(results):
    """
    Print evaluation metrics for all models
    """
    print("\nModel Evaluation Results:")
    print("-" * 80)
    for model_name, metrics in results.items():
        print(f"\n{model_name}:")
        print(f"MSE: {metrics['mse']:.4f}")
        print(f"RMSE: {metrics['rmse']:.4f}")
        print(f"MAE: {metrics['mae']:.4f}")
        print(f"R² Score: {metrics['r2']:.4f}")

def predict_water_requirements(model, scaler, y_scaler, features):
    """
    Make predictions using the best model
    """
    # Scale features
    scaled_features = scaler.transform(features)
    
    # Make prediction
    if isinstance(model, tf.keras.Model):
        scaled_pred = model.predict(scaled_features, verbose=0)
    else:
        scaled_pred = model.predict(scaled_features)
    
    # Inverse transform prediction
    prediction = y_scaler.inverse_transform(scaled_pred.reshape(-1, 1))
    
    return prediction.ravel()


In [63]:

# Train and evaluate models
print("Starting model training and evaluation...")
models, results = create_and_evaluate_models(X_train, X_test, y_train, y_test)
print_results(results)


Starting model training and evaluation...
Training Random Forest model...
Training Neural Network (MLPRegressor) model...
Training Deep Neural Network (TensorFlow) model...


c:\Users\ASUS VIVOBOOK\envs\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Model Evaluation Results:
--------------------------------------------------------------------------------

random_forest:
MSE: 0.0037
RMSE: 0.0612
MAE: 0.0098
R² Score: 0.9962

neural_network_sklearn:
MSE: 0.0008
RMSE: 0.0287
MAE: 0.0159
R² Score: 0.9992

deep_neural_network:
MSE: 0.0155
RMSE: 0.1246
MAE: 0.0871
R² Score: 0.9843


In [68]:
# Example of making predictions with the best model
# Find best model based on R² score
best_model_name = max(results.items(), key=lambda x: x[1]['r2'])[0]
best_model = models[best_model_name]

print(f"\nBest performing model: {best_model_name}")

# Example prediction using first few samples from test set
sample_features = X_test[10:15]

# Make predictions based on model type
if isinstance(best_model, tf.keras.Model):
    scaled_preds = best_model.predict(sample_features, verbose=0)
else:
    scaled_preds = best_model.predict(sample_features).reshape(-1, 1)

# Inverse transform predictions and actual values
predictions = y_scaler.inverse_transform(scaled_preds)
actual_values = y_scaler.inverse_transform(y_test[10:15].reshape(-1, 1))

print("\nSample Predictions (first 5 test samples):")
print("Predicted water requirements:", predictions.ravel())
print("Actual water requirements:", actual_values.ravel())


Best performing model: neural_network_sklearn

Sample Predictions (first 5 test samples):
Predicted water requirements: [219.08947948  50.85579111 185.82848669  10.39267439   0.40965692]
Actual water requirements: [2.2113e+02 5.1130e+01 1.8570e+02 1.0970e+01 6.0000e-02]
